Library

In [3]:
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd
import selenium

Get data

In [4]:
file_path = "../../Data/Club/all_data_of_clubs.csv"
data = pd.read_csv(file_path)
data.rename(columns={"Unnamed: 0": "Team"}, inplace=True)

Prepare attacking data

In [5]:
attacking_features = ['Goals', 'Goals per match', 'Penalties scored', 'Shots', 'Shots on target', 'Big Chances Created', 'Shooting accuracy %']
attacking_data = data[['Team'] + attacking_features]

Prepare defensive data

In [6]:
defensive_features = ['Clean sheets', 'Goals Conceded', 'Tackles', 'Tackle success %', 'Errors leading to goal', 'Interceptions']
defensive_data = data[['Team'] + defensive_features]

Prepare prossessing data

In [7]:
control_features = ['Passes', 'Passes per match', 'Pass accuracy %', 'Crosses', 'Cross accuracy %']
control_data = data[['Team'] + control_features]

control_fig = px.bar(
    control_data.melt(id_vars='Team', value_vars=control_features, var_name='Metric', value_name='Value'),
    x='Team', y='Value', color='Metric',
    barmode='group',
    title="Comparison of Control Metrics Across Teams"
)

Heatmap of Correlations - Attacking Performance

In [8]:
attack_data_corr = data[['Goals', 'Goals per match', 'Penalties scored', 'Shots', 'Shots on target',
                         'Big Chances Created', 'Shooting accuracy %', 'Passes', 'Passes per match',
                         'Pass accuracy %']].corr()

heatmap_data = attack_data_corr.reset_index().melt(id_vars='index')
heatmap_data.columns = ['Feature1', 'Feature2', 'Correlation']

heatmap_fig = px.imshow(
    attack_data_corr,
    x=attack_data_corr.columns,
    y=attack_data_corr.columns,
    zmin=-1, zmax=1,
    color_continuous_scale='RdBu',
    title="Correlation Heatmap of Attack Metrics",
    labels={'color': 'Correlation'}
)

Heatmap of Correlations - Defensing Performance

In [9]:
defense_data_corr = data[['Clean sheets', 'Goals Conceded', 'Tackles', 'Tackle success %', 'Interceptions', 'Saves',
                          'Blocked shots', 'Clearances', 'Headed Clearance', 'Aerial Battles/Duels Won', 'Own goals',
                          'Passes', 'Passes per match', 'Pass accuracy %']].corr()

defense_heatmap_fig = px.imshow(
    defense_data_corr,
    x=defense_data_corr.columns,
    y=defense_data_corr.columns,
    zmin=-1, zmax=1,
    color_continuous_scale='RdBu',
    title="Correlation Heatmap of Defensive Metrics",
    labels={'color': 'Correlation'}
)

Yellow Cards and Fouls

In [10]:
scatter_fig = px.scatter(
    data_frame=data,
    x='Fouls',
    y='Yellow cards',
    title="Scatter Plot of Fouls vs. Yellow Cards",
    labels={'Fouls': 'Fouls', 'Yellow cards': 'Yellow Cards'},
    hover_name='Team',
    template="plotly",
    height=600,  # Set height to ensure better display
)

Red Cards and Fouls

In [11]:
scatter_red_fig = px.scatter(
    data_frame=data,
    x='Fouls',
    y='Red cards',
    title="Scatter Plot of Fouls vs. Red Cards",
    labels={'Fouls': 'Fouls', 'Red cards': 'Red Cards'},
    hover_name='Team',
    template="plotly",
    height=600,  # Set height for better visualization
)


Layout

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1("Football Clubs Performance Metrics", style={'textAlign': 'center'}),

    html.Div([
        html.H2("Attacking Metrics Trends", style={'textAlign': 'center'}),
        dcc.Graph(
            figure=px.line(
                attacking_data.melt(id_vars='Team', value_vars=attacking_features, var_name='Metric', value_name='Value'),
                x='Team', y='Value', color='Metric',
                title="Attacking Metrics Trends Across Teams"
            )
        )
    ]),

    html.Div([
        html.H2("Defensive Metrics Stacked Bar", style={'textAlign': 'center'}),
        dcc.Graph(
            figure=px.bar(
                defensive_data.melt(id_vars='Team', value_vars=defensive_features, var_name='Metric', value_name='Value'),
                x='Team', y='Value', color='Metric',
                title="Defensive Metrics Distribution Across Teams",
                barmode='stack'
            )
        )
    ]),
    
    html.Div([
        html.H2("Control Metrics Grouped Bar Chart", style={'textAlign': 'center'}),
        dcc.Graph(figure=control_fig)
    ]),
    
    html.Div([
        html.H2("Correlation Heatmap of Attack Metrics", style={'textAlign': 'center'}),
        dcc.Graph(figure=heatmap_fig)
    ]),
    
    html.Div([
        html.H2("Correlation Heatmap of Defensive Metrics", style={'textAlign': 'center'}),
        dcc.Graph(figure=defense_heatmap_fig)
    ]),

    html.Div([
        html.H2("Scatter Plot: Fouls vs. Yellow Cards", style={'textAlign': 'center'}),
        dcc.Graph(figure=scatter_fig)
    ]),
    
    html.Div([
        html.H2("Scatter Plot: Fouls vs. Red Cards", style={'textAlign': 'center'}),
        dcc.Graph(figure=scatter_red_fig)
    ])

])

app.run_server(mode='external', debug=True)